### 训练神经网络
* 神经网络的典型训练过程如下：

1. 定义包含一些可学习的参数(或者叫权重)神经网络模型；
2. 在数据集上迭代；
3. 通过神经网络处理输入；
4. 计算损失(输出结果和正确值的差值大小)；
5. 将梯度反向传播回网络的参数；
6. 更新网络的参数，主要使用如下简单的更新原则： weight = weight - learning_rate * gradient


#### 定义网络

* 在模型中必须要定义 forward 函数，backward 函数（用来计算梯度）会被autograd自动创建。 可以在 forward 函数中使用任何针对 Tensor 的操作

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input , 6 output , 5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 线性激活函数
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling (2,2) 池化层，选择最大
        X = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        X = F.max_pool2d(F.relu(self.conv2(x)), 2)
        X = x.view(-1, self.num_flat_features(x))
        X = F.relu(self.fc1(x))
        X = F.relu(self.fc2(x))
        X = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = X.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
# net.parameters()返回可被学习的参数列表和值
params = list(net.parameters())
print(len(params))
# print(params)
print(params[0].size())  #第一层的权重

10
torch.Size([6, 1, 5, 5])


In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

### 损失函数

In [ ]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()       # 均方误差

loss = criterion(output, target)
print(loss)

### 反向传播
* 但是在调用前需要清除已存在的梯度，否则梯度将被累加到已存在的梯度

In [ ]:
# 清除梯度
net.zero_grad()

print(net.conv1.bias.grad)
# 反向传播的误差
loss.backward()
print(net.conv1.bias.grad)

### 更新权重

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_ratea)

* 但是当使用神经网络是想要使用各种不同的更新规则时，比如SGD、Nesterov-SGD、Adam、RMSPROP等，PyTorch中构建了一个包torch.optim实现了所有的这些规则

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_gradr()
loss = criterion(output, target)
loss.backward()
optimizer.step()